### Tools
Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:
1. A schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema)
2. A function or coroutine to execute.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq

model = ChatGroq(model="qwen/qwen3-32b")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000021609FF41A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021609FF4EC0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
response = model.invoke("Who is KSI?")
print(response.content)

<think>
Okay, so I need to figure out who KSI is. Let me start by recalling what I know. I think KSI is a YouTuber and a boxer, right? He's British, I believe. He's been in the news for his boxing matches, maybe even against other YouTubers like Logan Paul or Jake Paul. But I'm not entirely sure about the details.

First, his real name. I think his full name is Olajide Olatunji. Yeah, that sounds familiar. He was a YouTuber with a channel where he did comedy sketches and maybe some challenges. He gained a lot of followers over the years. Then, he started boxing. I remember he had a big match against Jake Paul in 2019. That was a big deal because both were YouTubers with massive followings, and it was a crossover event between YouTube and boxing. The fight was pretty intense, and KSI won, right? Or did he lose? Wait, I think Jake Paul won that one. But KSI might have won another fight later. I need to check that.

Also, KSI is part of the "Boxing Socialites" or something like that, wher

In [5]:
from langchain.tools import tool

@tool
def get_weather(city: str)->str:
    """Get the weather of a city"""
    return f"The weather of {city} is sunny"

In [6]:
model_with_tool = model.bind_tools([get_weather])
model_with_tool

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000021609FF41A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021609FF4EC0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_weather', 'description': 'Get the weather of a city', 'parameters': {'properties': {'city': {'type': 'string'}}, 'required': ['city'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [14]:
response = model_with_tool.invoke("What is the weather in Delhi?")

print(response)
for tool_call in response.tool_calls:
    print(f"Tool : {tool_call['name']}")
    print(f"Args : {tool_call['args']}")

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Delhi. Let me check the tools available. There\'s a function called get_weather that takes a city parameter. Since the user specified Delhi, I need to call that function with "Delhi" as the city argument. I\'ll make sure the parameters are correctly formatted in JSON and return the tool call as specified.\n', 'tool_calls': [{'id': 'p26bw5z79', 'function': {'arguments': '{"city":"Delhi"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 153, 'total_tokens': 251, 'completion_time': 0.144504352, 'completion_tokens_details': {'reasoning_tokens': 73}, 'prompt_time': 0.006154735, 'prompt_tokens_details': None, 'queue_time': 0.064882305, 'total_time': 0.150659087}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provid

### Tool Execution Loops

In [15]:
# Step 1 : Model generates tool calls
messages = [{"role" : "user", "content" : "What is the weather in Delhi?"}]
ai_msg = model_with_tool.invoke(messages)
messages.append(ai_msg)
messages

[{'role': 'user', 'content': 'What is the weather in Delhi?'},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Delhi. Let me check the tools available. There\'s a function called get_weather that takes a city parameter. Since the user specified Delhi, I need to call that function with "Delhi" as the city argument. I\'ll make sure the JSON is correctly formatted with the city name in the parameters.\n', 'tool_calls': [{'id': '1g093y3gv', 'function': {'arguments': '{"city":"Delhi"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 153, 'total_tokens': 249, 'completion_time': 0.144460202, 'completion_tokens_details': {'reasoning_tokens': 71}, 'prompt_time': 0.007060101, 'prompt_tokens_details': None, 'queue_time': 0.079054798, 'total_time': 0.151520303}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'f

In [16]:
# Step 2 : Execute tool and collect results
for tool_call in ai_msg.tool_calls:
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

messages

[{'role': 'user', 'content': 'What is the weather in Delhi?'},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Delhi. Let me check the tools available. There\'s a function called get_weather that takes a city parameter. Since the user specified Delhi, I need to call that function with "Delhi" as the city argument. I\'ll make sure the JSON is correctly formatted with the city name in the parameters.\n', 'tool_calls': [{'id': '1g093y3gv', 'function': {'arguments': '{"city":"Delhi"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 153, 'total_tokens': 249, 'completion_time': 0.144460202, 'completion_tokens_details': {'reasoning_tokens': 71}, 'prompt_time': 0.007060101, 'prompt_tokens_details': None, 'queue_time': 0.079054798, 'total_time': 0.151520303}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'f

In [19]:
# Step 3 : Pass results back to the model for final response
final_response = model_with_tool.invoke(messages)
print(final_response.content)

The weather in Delhi is currently sunny. A perfect day to enjoy outdoor activities! 😊
